In [6]:
import pandas as pd
import operator
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def calculateSimilarity(sentence, document):
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform([sentence] + document)
    similarity_matrix = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:])
    return similarity_matrix[0][0]

data = pd.read_csv('lda_content_10000.csv')

# Assuming 'processed_content' contains the sentences to summarize
sentences = data['processed_content'].tolist()

scores = {}
for sentence in sentences:
    temp_doc = [s for s in sentences if s != sentence]
    score = calculateSimilarity(sentence, temp_doc)
    scores[sentence] = score

n = int(20 * len(sentences) / 100)
alpha = 0.5
summarySet = []

while n > 0:
    mmr = {}
    
    for sentence in scores.keys():
        if sentence not in summarySet:
            mmr[sentence] = alpha * scores[sentence] - (1 - alpha) * calculateSimilarity(sentence, summarySet)
    
    selected = max(mmr.items(), key=operator.itemgetter(1))[0]
    summarySet.append(selected)
    n -= 1

print('Ringkasan Artikel:\n')

for sentence in summarySet:
    print(sentence.lstrip(' '))
    print()
